## PAIR PROGRAMMING ETL III

### ETL Transformación II - Clases y Funciones de limpieza
---

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [48]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import ast

In [152]:
class Ataquesclima:

    def __init__(self, dicc_datos):

        self.dicc_datos= dicc_datos

    def conexion_API(self, producto):
        
        # Especificamos el producto del que queremos extraer la información
        self.producto= producto
       
        # Creamos un dataframe vacío con las columnas que rellenaremos con la información obtenida de la API
        df_clima_paises = pd.DataFrame( columns= ['timepoint', 'cloudcover', 'lifted_index', 'prec_type', 'prec_amount','temp2m', 'rh2m', 'weather', 'wind10m.direction', 'wind10m.speed','country'])

        for key, value in self.dicc_datos.items(): 

            url =f'http://www.7timer.info/bin/api.pl?lon=-{self.dicc_datos[key][0]}&lat={self.dicc_datos[key][1]}&product={self.producto}&output=json'

            response = requests.get(url=url)
            codigo_estado = response.status_code
            razon_estado = response.reason
            if codigo_estado == 200:
                print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 402:
                print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 404:
                print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            else:
                print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)

            # Convetimos la información obtenida en formato json a un dataframe
            df = pd.json_normalize(response.json()['dataseries']) 
            # Creamos una nueva columna para que inserte la key que corresponde al nombre del país
            df['country']=key.lower()
            # Concatenamos los dataframes de cada país en uno
            df_clima_paises=pd.concat([df_clima_paises,df], axis=0, ignore_index = True)
            return df_clima_paises


    def limpiar_meteo(self, df_clima_paises): 

        self.df_clima_paises = df_clima_paises
        
        #df_clima_paises['rh_profile']= df_clima_paises['rh_profile'].apply(ast.literal_eval)
        # Para separar la lista de diccionarios en varias columnas
        x = df_clima_paises['rh_profile'].apply(pd.Series)     

        # For loop para sacar el nombre de la columna y los valores de las filas
        for i in range(len(x.columns)): 
        
            # aplicamos el apply,extraemos el valor de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 
            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(x[i].apply(pd.Series)["rh"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_clima_paises.insert(i, nombre, valores)
      
            
        #['wind_profile']= df_clima_paises['wind_profile'].apply(ast.literal_eval)
        y = df_clima_paises['wind_profile'].apply(pd.Series)

            # For loop para sacar el nombre de la columna y los valores de las filas
        for i in range(len(y.columns)): 
                
            # aplicamos el apply,extraemos los valores de la key "layer" y lo almacenamos en dos variables que convertimos a strings
            nombre = "direction" + str(y[i].apply(pd.Series)["layer"][0]) 
            nombre2 = "speed" + str(y[i].apply(pd.Series)["layer"][0]) 

            # hacemos lo mismo con dos variables para "guardar" los valores
            valores = list(y[i].apply(pd.Series)["direction"] )
            valores2= list(y[i].apply(pd.Series)["speed"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_clima_paises.insert(i, nombre, valores)
            df_clima_paises.insert(i,nombre2,valores2)
    

            # Eliminamos las columnas que tienen las listas de diccionarios, información duplicada
            df_clima_paises.drop(['rh_profile','wind_profile'], axis=1, inplace=True)

            #Calculamos la media por pais
            df_clima_paises = df_clima_paises.groupby('country').mean()

            #Calculamos la fecha del dato y añadimos la columna al dataframe
            hoy = datetime.now()
            hoy = datetime.strftime(hoy, '%Y-%m-%d')
            df_clima_paises["fecha"] = hoy

            #reseteamos el indice para guardar correctamente el df.
            df_clima_paises.reset_index(inplace=True)
            
            print(f'Ya está listo el dataframes de meteo para juntarlo con el de ataques')
            return df_clima_paises
            

    def juntar_dfs(self, df_attacks, df_clima_paises): 
        
        self.df_attacks = df_attacks
        self.df_clima_paises = df_clima_paises

        df_union= df_attacks.merge(df_clima_paises, how= 'inner', on= 'country')


        print("El df de union tiene las siguientes columnas: \n", list(df_union.columns))
      
        # guardamos los datos
        df_union.to_pickle('../files/datos_actualizados.pkl')
        df_union.to_csv('../files/datos_actualizados.csv')

        return df_union

    def chequear_datos(self, df_union): 

        print(f"El número de filas son: {df_union.shape[0]}\ny el número de columnas son: {df_union.shape[1]}")
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df_union.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df_union.isnull().sum() / df_union.shape[0]) *  100)



            

In [125]:
 # Hacemos un diccionario con los países como keys y las coordenadas como values
dicc_datos= {'usa': [-100.445882, 39.7837304],'australia': [134.755, -24.7761086],'south africa': [24.991639, -28.8166236],'new zealand': [172.8344077, -41.5000831],'papua new guinea': [144.2489081, -5.6816069]}

In [153]:
api = Ataquesclima(dicc_datos) # Iniciamos la clase

In [143]:
df_clima_paises= api.conexion_API('meteo') # Hacemos la llamada a la API
df_clima_paises.head(2)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,country,highcloud,midcloud,lowcloud,rh_profile,wind_profile,msl_pressure,snow_depth
0,3,7,6,none,0,16,4,NaN,110,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 150, 'speed':...",1024.0,0.0
1,6,8,6,none,0,15,6,NaN,220,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 265, 'speed':...",1025.0,0.0


In [144]:
df_clima_paises = api.limpiar_meteo(df_clima_paises)


Ya está listo el dataframes de meteo para juntarlo con el de ataques


/tmp/ipykernel_2489/1731046026.py:83: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_clima_paises = df_clima_paises.groupby('country').mean()


In [145]:
df_clima_paises.head()

,country,speed950mb,direction950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,...,rh_350mb,rh_300mb,rh_250mb,rh_200mb,highcloud,midcloud,lowcloud,msl_pressure,snow_depth,fecha
0,usa,2.875,234.6875,4.375,5.140625,5.65625,4.71875,3.1875,2.59375,2.296875,...,5.3125,6.140625,7.296875,5.984375,-9999.0,-9999.0,-9999.0,1019.5625,0.0,2023-01-11


In [146]:
df_attacks = pd.read_pickle("../files/attacks12.pickle")

In [154]:
df_union= api.juntar_dfs(df_attacks, df_clima_paises)

El df de union tiene las siguientes columnas: 
 ['case_number', 'type', 'country', 'activity', 'age', 'species_', 'date', 'mes_ataque', 'fatal', 'sexo', 'cat_species', 'mes', 'year', 'edades', 'speed950mb', 'direction950mb', 'rh_950mb', 'rh_900mb', 'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb', 'rh_550mb', 'rh_500mb', 'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb', 'rh_250mb', 'rh_200mb', 'highcloud', 'midcloud', 'lowcloud', 'msl_pressure', 'snow_depth', 'fecha']


In [155]:
api.chequear_datos(df_union)

El número de filas son: 741
y el número de columnas son: 38
-----------------------------------------
Los tipos de datos que tenemos son: 

case_number        object
type               object
country            object
activity           object
age                object
species_           object
date               object
mes_ataque         object
fatal              object
sexo               object
cat_species        object
mes                object
year              float64
edades            float64
speed950mb        float64
direction950mb    float64
rh_950mb          float64
rh_900mb          float64
rh_850mb          float64
rh_800mb          float64
rh_750mb          float64
rh_700mb          float64
rh_650mb          float64
rh_600mb          float64
rh_550mb          float64
rh_500mb          float64
rh_450mb          float64
rh_400mb          float64
rh_350mb          float64
rh_300mb          float64
rh_250mb          float64
rh_200mb          float64
highcloud         float64
mi